<pre>
1. Download the data from <a href='https://drive.google.com/file/d/15dCNcmKskcFVjs7R0ElQkR61Ex53uJpM/view?usp=sharing'>here</a>

2. Code the model to classify data like below image

<img src='https://i.imgur.com/33ptOFy.png'>

3. Write your own callback function, that has to print the micro F1 score and AUC score after each epoch.

4. Save your model at every epoch if your validation accuracy is improved from previous epoch. 

5. you have to decay learning based on below conditions 
        Cond1. If your validation accuracy at that epoch is less than previous epoch accuracy, you have to decrese the
               learning rate by 10%. 
        Cond2. For every 3rd epoch, decay your learning rate by 5%.
        
6. If you are getting any NaN values(either weigths or loss) while training, you have to terminate your training. 

7. You have to stop the training if your validation accuracy is not increased in last 2 epochs.

8. Use tensorboard for every model and analyse your gradients. (you need to upload the screenshots for each model for evaluation)

9. use cross entropy as loss function

10. Try the architecture params as given below. 
</pre>

<pre>
<b>Model-1</b>
<pre>
1. Use tanh as an activation for every layer except output layer.
2. use SGD with momentum as optimizer.
3. use RandomUniform(0,1) as initilizer.
3. Analyze your output and training process. 
</pre>
</pre>
<pre>
<b>Model-2</b>
<pre>
1. Use relu as an activation for every layer except output layer.
2. use SGD with momentum as optimizer.
3. use RandomUniform(0,1) as initilizer.
3. Analyze your output and training process. 
</pre>
</pre>
<pre>
<b>Model-3</b>
<pre>
1. Use relu as an activation for every layer except output layer.
2. use SGD with momentum as optimizer.
3. use he_uniform() as initilizer.
3. Analyze your output and training process. 
</pre>
</pre>
<pre>
<b>Model-4</b>
<pre>
1. Try with any values to get better accuracy/f1 score.  
</pre>
</pre>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Importing libraries

In [ ]:
# tf.enable_eager_execution()
import os

In [ ]:
# !pip install tensorflow-addons

In [ ]:
# import tensorflow_addons as tfa

In [ ]:
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense,Input,Activation
from tensorflow.keras.models import Model
import random as rn
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.callbacks import Callback
from sklearn.metrics import f1_score

#Reading and splitting data

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Datasets/data.csv')

In [ ]:
df.head()

,f1,f2,label
0,0.450564,1.074305,0.0
1,0.085632,0.967682,0.0
2,0.117326,0.971521,1.0
3,0.982179,-0.380408,0.0
4,-0.720352,0.955850,0.0


In [ ]:
df['label'].value_counts()

1.0    10000
0.0    10000
Name: label, dtype: int64

In [ ]:
x = df.drop('label',axis=1)
y = df['label']

In [ ]:
x.shape,y.shape

((20000, 2), (20000,))

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2,random_state=42)

In [ ]:
x_train.shape,x_test.shape

((16000, 2), (4000, 2))

#MODEL 1 - tanh

In [ ]:
K.clear_session()

In [ ]:
if not os.path.isdir('model_save_sgd'):
    os.mkdir('model_save_sgd')

In [ ]:
#Input layer
input_layer = Input(shape=(x_train.shape[1],))
#Dense hidden layer 1
layer1 = Dense(256,activation='tanh',kernel_initializer=tf.keras.initializers.RandomUniform(0,1))(input_layer)
#Dense hidden layer 2
layer2 = Dense(128,activation='tanh',kernel_initializer=tf.keras.initializers.RandomUniform(0,1))(layer1)
#Dense hidden layer 3
layer3 = Dense(128,activation='tanh',kernel_initializer=tf.keras.initializers.RandomUniform(0,1))(layer2)
#Dense hidden layer 4
layer4 = Dense(64,activation='tanh',kernel_initializer=tf.keras.initializers.RandomUniform(0,1))(layer3)
#Dense hidden layer 5
layer5 = Dense(32,activation='tanh',kernel_initializer=tf.keras.initializers.RandomUniform(0,1))(layer4)
#output layer
output = Dense(1,activation='sigmoid',kernel_initializer=tf.keras.initializers.RandomUniform(0,1))(layer5)
#Creating a model
model = Model(inputs=input_layer,outputs=output)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 2)]               0         
_________________________________________________________________
dense (Dense)                (None, 256)               768       
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_3 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_4 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 33    

In [ ]:
class TerminateNaN(tf.keras.callbacks.Callback):
        
    def on_epoch_end(self, epoch, logs={}):
        # print(self.model.validation_data[0])
        loss = logs.get('loss')
        if loss is not None:
            if np.isnan(loss) or np.isinf(loss):
                print("Invalid loss and terminated at epoch {}".format(epoch))
                self.model.stop_training = True

In [ ]:
def changeLearningRate(epoch,lr):
    # initial_learningrate=0.1
    # print(epoch)
    if (epoch+1) % 3 == 0:
        lr = lr - lr*(0.05)
    return lr

In [ ]:
# class Metrics(Callback):
#     # def on_train_begin(self, logs={}):
#         # self.val_f1 = []
#         # self.val_recalls = []
#         # self.val_precisions = []
#     def on_epoch_end(self, epoch, logs={}):
#         # val_predict = (np.asarray(self.model.predict(x_test))).round()
#         # val_targ = y_test#self.model.validation_data[1]
#         # _val_f1 = f1_score(val_targ, val_predict)
#         # # _val_recall = recall_score(val_targ, val_predict)
#         # # _val_precision = precision_score(val_targ, val_predict)
#         # self.val_f1.append(_val_f1)
#         # self.val_recalls.append(_val_recall)
#         # self.val_precisions.append(_val_precision)
#         # print “ — val_f1: %f — val_precision: %f — val_recall %f” %(_val_f1, _val_precision, _val_recall)
#         logs['f1'] = 2 * (logs['pr'] * logs['re']) / (logs['pr'] + logs['re'])
#         print(f'F1 score : ',logs['f1'])
#         return
# metrics = Metrics()

##Custom F1 score

In [ ]:
def f1(y_true,y_pred):
    # print(y_pred)
    y_pred = K.round(K.clip(y_pred, 0, 1))
    return tf.py_function(f1_score,(y_true,y_pred),tf.double)

In [ ]:
# def f1(y_true, y_pred):
    # y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    # y_pred_neg = 1 - y_pred_pos

    # y_pos = K.round(K.clip(y_true, 0, 1))
    # y_neg = 1 - y_pos

    # tp = K.sum(y_pos * y_pred_pos)
    # tn = K.sum(y_neg * y_pred_neg)

    # fp = K.sum(y_neg * y_pred_pos)
    # fn = K.sum(y_pos * y_pred_neg)

    # pr = tp / (tp + fp)
    # re = tp / (tp + fn)

    # f1 = (2*pr*re) / (pr + re)
    # print(type(y_true.eval(session=None)))


    # numerator = (tp * tn - fp * fn)
    # denominator = K.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))

    # return 1#f1_score(y_true, y_pred)

In [ ]:
# class LossHistory(tf.keras.callbacks.Callback):
    
#     def on_train_begin(self, logs={}):
#         ## on begin of training, we are creating a instance varible called history
#         ## it is a dict with keys [loss, acc, val_loss, val_acc]
#         self.history={'loss': [],'acc': [],'val_loss': [],'val_acc': [],'f1':[], 'auc' :[], 'precision':[], 'recall':[]}
        
#     def on_epoch_end(self, epoch, logs={}):
#         ## on end of each epoch, we will get logs and update the self.history dict
#         print(logs)
#         self.history['loss'].append(logs.get('loss'))
#         self.history['acc'].append(logs.get('acc'))
#         if logs.get('val_loss', -1) != -1:
#             self.history['val_loss'].append(logs.get('val_loss'))
#         if logs.get('val_acc', -1) != -1:
#             self.history['val_acc'].append(logs.get('val_acc'))
#         if logs.get('auc', -1) != -1:
#             self.history['auc'].append(logs.get('auc'))
        # if (logs.get('precision', -1) != -1) and (logs.get('recall', -1) != -1):
        #     f1 = 2*((logs.get('precision') * logs.get('recall'))/ (logs.get('precision') + logs.get('recall')))
        #     self.history['f1'].append(f1)
            
# history_own=LossHistory() 

##Callbacks

In [ ]:

log_dir="/content/model_save_sgd/" 
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1, write_graph=True,write_grads=True)

filepath="model_save_sgd/weights-{epoch:02d}.hdf5"
checkpoint = ModelCheckpoint(filepath,monitor='val_accuracy',verbose=1)  #,save_best_only=True
# validation_accuracy learning rate
val_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.9,patience=0)
# lr change based on no.of epoch
lrschedule = LearningRateScheduler(changeLearningRate, verbose=1)
#earlystopping
earlystop = EarlyStopping(monitor='val_accuracy', patience=2, verbose=1)
#terminate on nan
term_nan = TerminateNaN()
# here we are creating a list with all the callbacks we want
callback_list = [val_lr,lrschedule, earlystop, checkpoint,term_nan,tensorboard_callback]    #,metrics

In [ ]:
opt = tf.keras.optimizers.SGD(learning_rate=10,momentum=0.9)

In [ ]:

        # for layer in self.model.layers:
        #     if np.nan in layer:
        #         self.model.stop_training = True

# lambda_call = LambdaCallback(on_epoch_end= lambda epochs, logs: model.stop_training = True if np.nan in np.ravel(np.array([layer.get_weights() for layer in model.layers])))
# model.compile(optimizer=opt, loss='categorical_crossentropy',metrics=['accuracy',tf.keras.metrics.AUC(),tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])

In [ ]:
# model.compile(optimizer=opt, loss='binary_crossentropy',metrics=['accuracy',tf.keras.metrics.AUC(),tf.keras.metrics.Precision(name='pr'),tf.keras.metrics.Recall(name='re')])#,tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])

In [ ]:

model.compile(optimizer=opt, loss='binary_crossentropy',metrics=['accuracy',tf.keras.metrics.AUC(),f1])

In [ ]:
history = model.fit(x_train,y_train,epochs=10, validation_data=(x_test,y_test),batch_size=16, callbacks=callback_list)

Epoch 1/10

Epoch 00001: LearningRateScheduler setting learning rate to 10.0.
1000/1000 [==============================] - 40s 5ms/step - loss: 101.8881 - accuracy: 0.5009 - auc_1: 0.5013 - f1: 0.4673 - val_loss: 68.8892 - val_accuracy: 0.5110 - val_auc_1: 0.5110 - val_f1: 0.4906

Epoch 00001: saving model to model_save_sgd/weights-01.hdf5
Epoch 2/10

Epoch 00002: LearningRateScheduler setting learning rate to 10.0.
1000/1000 [==============================] - 6s 6ms/step - loss: 105.8496 - accuracy: 0.5031 - auc_1: 0.5026 - f1: 0.4736 - val_loss: 323.8675 - val_accuracy: 0.4890 - val_auc_1: 0.4890 - val_f1: 0.4717

Epoch 00002: saving model to model_save_sgd/weights-02.hdf5
Epoch 3/10

Epoch 00003: LearningRateScheduler setting learning rate to 8.55.
1000/1000 [==============================] - 5s 5ms/step - loss: 80.8388 - accuracy: 0.4964 - auc_1: 0.4958 - f1: 0.4632 - val_loss: 13.6245 - val_accuracy: 0.4890 - val_auc_1: 0.4890 - val_f1: 0.4717

Epoch 00003: saving model to model_s

In [ ]:
# model.layers[1].get_weights()
# model.add_metric('f1')

##Tensorboard

In [ ]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [3]:
# %tensorboard --logdir /content/model_save_sgd

In [ ]:
history.history

{'accuracy': [0.4958750009536743, 0.5074999928474426, 0.4988124966621399],
 'auc': [0.4961993396282196, 0.5093865394592285, 0.4988124966621399],
 'f1': [0.4726143479347229, 0.48473894596099854, 0.4819086194038391],
 'loss': [41.09354782104492, 39.81825256347656, 99.82398223876953],
 'lr': [10, 1, 0],
 'val_accuracy': [0.5115000009536743, 0.5115000009536743, 0.5115000009536743],
 'val_auc': [0.5114981532096863, 0.5114981532096863, 0.5114981532096863],
 'val_f1': [0.4919758439064026, 0.4919758439064026, 0.4919758439064026],
 'val_loss': [64.7737808227539, 97.29852294921875, 97.29852294921875]}

#MODEL - 2 (relu)

In [ ]:
K.clear_session()

In [ ]:
if not os.path.isdir('model_save_relu'):
    os.mkdir('model_save_relu')

In [ ]:
#Input layer
input_layer = Input(shape=(x_train.shape[1],))
#Dense hidden layer 1
layer1 = Dense(64,activation='relu',kernel_initializer=tf.keras.initializers.RandomUniform(0,1))(input_layer)
#Dense hidden layer 2
layer2 = Dense(128,activation='relu',kernel_initializer=tf.keras.initializers.RandomUniform(0,1))(layer1)
#Dense hidden layer 3
layer3 = Dense(256,activation='relu',kernel_initializer=tf.keras.initializers.RandomUniform(0,1))(layer2)
#Dense hidden layer 4
layer4 = Dense(128,activation='relu',kernel_initializer=tf.keras.initializers.RandomUniform(0,1))(layer3)
#Dense hidden layer 5
layer5 = Dense(64,activation='relu',kernel_initializer=tf.keras.initializers.RandomUniform(0,1))(layer4)
#output layer
output = Dense(1,activation='sigmoid',kernel_initializer=tf.keras.initializers.RandomUniform(0,1))(layer5)
#Creating a model
model1 = Model(inputs=input_layer,outputs=output)

In [ ]:
# model1.compile(optimizer=opt, loss='binary_crossentropy',metrics=['accuracy',tf.keras.metrics.AUC(),tf.keras.metrics.Precision(name='pr'),tf.keras.metrics.Recall(name='re')])#,tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])

In [ ]:
opt = tf.keras.optimizers.SGD(learning_rate=10,momentum=0.9)

##Callbacks

In [ ]:

log_dir="/content/model_save_relu/" 
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1, write_graph=True,write_grads=True)

filepath="model_save_relu/weights-{epoch:02d}.hdf5"
checkpoint = ModelCheckpoint(filepath,monitor='val_accuracy',verbose=1)  #,save_best_only=True
# validation_accuracy learning rate
val_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.9,patience=0)
# lr change based on no.of epoch
lrschedule = LearningRateScheduler(changeLearningRate, verbose=1)
#earlystopping
earlystop = EarlyStopping(monitor='val_accuracy', patience=2, verbose=1)
#terminate on nan
term_nan = TerminateNaN()
# here we are creating a list with all the callbacks we want
callback_list = [val_lr,lrschedule, earlystop, checkpoint,term_nan,tensorboard_callback]

In [ ]:
model1.compile(optimizer=opt, loss='binary_crossentropy',metrics=['accuracy',tf.keras.metrics.AUC(),f1])

In [ ]:
history1 = model1.fit(x_train,y_train,epochs=10, validation_data=(x_test,y_test),batch_size=16, callbacks=callback_list)

Epoch 1/10

Epoch 00001: LearningRateScheduler setting learning rate to 10.0.
1000/1000 [==============================] - 6s 5ms/step - loss: 44827.0781 - accuracy: 0.4912 - auc: 0.4916 - f1: 0.3282 - val_loss: 0.7253 - val_accuracy: 0.5008 - val_auc: 0.5000 - val_f1: 0.0000e+00

Epoch 00001: saving model to model_save_relu/weights-01.hdf5
Epoch 2/10

Epoch 00002: LearningRateScheduler setting learning rate to 10.0.
1000/1000 [==============================] - 5s 5ms/step - loss: 2.2467 - accuracy: 0.5038 - auc: 0.5019 - f1: 0.3303 - val_loss: 2.1571 - val_accuracy: 0.5008 - val_auc: 0.5000 - val_f1: 0.0000e+00

Epoch 00002: saving model to model_save_relu/weights-02.hdf5
Epoch 3/10

Epoch 00003: LearningRateScheduler setting learning rate to 8.55.
1000/1000 [==============================] - 5s 5ms/step - loss: 1.4357 - accuracy: 0.5002 - auc: 0.5012 - f1: 0.3333 - val_loss: 1.2444 - val_accuracy: 0.5008 - val_auc: 0.5000 - val_f1: 0.0000e+00

Epoch 00003: saving model to model_save_

##Tensorboard

In [2]:
# %tensorboard --logdir /content/model_save_relu

In [ ]:
history1.history

{'accuracy': [0.49806249141693115, 0.49668750166893005, 0.4961875081062317],
 'auc_3': [0.49951624870300293, 0.4971032738685608, 0.49379390478134155],
 'f1': [0.3242529034614563, 0.32708629965782166, 0.3294018805027008],
 'loss': [1.7061606645584106, 1.6221518516540527, 1.3496028184890747],
 'lr': [10, 9, 7],
 'val_accuracy': [0.49924999475479126,
  0.49924999475479126,
  0.49924999475479126],
 'val_auc_3': [0.5, 0.5, 0.5],
 'val_f1': [0.6553722023963928, 0.6553722023963928, 0.6553722023963928],
 'val_loss': [1.1186678409576416, 0.8830900192260742, 0.693648636341095]}

#MODEL - 3 (relu + he)

In [ ]:
from keras.callbacks import LambdaCallback


In [ ]:
K.clear_session()

In [ ]:
if not os.path.isdir('model_save_relu_he'):
    os.mkdir('model_save_relu_he')

In [ ]:
#Input layer
input_layer = Input(shape=(x_train.shape[1],))
#Dense hidden layer 1
layer1 = Dense(64,activation='relu',kernel_initializer=tf.keras.initializers.HeNormal())(input_layer)
#Dense hidden layer 2
layer2 = Dense(128,activation='relu',kernel_initializer=tf.keras.initializers.HeNormal())(layer1)
#Dense hidden layer 3
layer3 = Dense(256,activation='relu',kernel_initializer=tf.keras.initializers.HeNormal())(layer2)
#Dense hidden layer 4
layer4 = Dense(128,activation='relu',kernel_initializer=tf.keras.initializers.HeNormal())(layer3)
#Dense hidden layer 5
layer5 = Dense(64,activation='relu',kernel_initializer=tf.keras.initializers.HeNormal())(layer4)
#output layer
output = Dense(1,activation='sigmoid',kernel_initializer=tf.keras.initializers.RandomUniform(0,1,seed=30))(layer5)
#Creating a model
model2 = Model(inputs=input_layer,outputs=output)

In [ ]:
# print_weights = LambdaCallback(on_train_batch_begin=lambda batch, logs: print(model.layers[0].get_weights()))


##Callbacks

In [ ]:
log_dir="/content/model_save_relu_he/" 
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1, write_graph=True,write_grads=True)

filepath="model_save_relu_he/weights-{epoch:02d}.hdf5"
checkpoint = ModelCheckpoint(filepath,monitor='val_accuracy',verbose=1)  #,save_best_only=True
# validation_accuracy learning rate
val_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.9,patience=0)
# lr change based on no.of epoch
lrschedule = LearningRateScheduler(changeLearningRate, verbose=1)
#earlystopping
earlystop = EarlyStopping(monitor='val_accuracy', patience=2, verbose=1)
#terminate on nan
term_nan = TerminateNaN()
# here we are creating a list with all the callbacks we want
callback_list = [val_lr,lrschedule, earlystop, checkpoint,term_nan,tensorboard_callback]

In [ ]:
# model2.compile(optimizer=opt, loss='binary_crossentropy',metrics=['accuracy',tf.keras.metrics.AUC(),tf.keras.metrics.Precision(name='pr'),tf.keras.metrics.Recall(name='re')])#,tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])

In [ ]:
opt = tf.keras.optimizers.SGD(1,momentum=0.9)

In [ ]:
model2.compile(optimizer=opt, loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
history2 = model2.fit(x_train,y_train,epochs=10, validation_data=(x_test,y_test),batch_size=16, callbacks=callback_list)

Epoch 1/10

Epoch 00001: LearningRateScheduler setting learning rate to 1.0.
1000/1000 [==============================] - 3s 3ms/step - loss: nan - accuracy: 0.4998 - val_loss: nan - val_accuracy: 0.5008

Epoch 00001: saving model to model_save_relu_he/weights-01.hdf5
Invalid loss and terminated at epoch 0


In [ ]:
history2.history

{'accuracy': [0.49918749928474426],
 'loss': [nan],
 'lr': [1],
 'val_accuracy': [0.5007500052452087],
 'val_loss': [nan]}

Terminate on NaN is triggered

#Best Model ( RELU + HE init + ADAM opt )

In [ ]:
K.clear_session()

In [ ]:
if not os.path.isdir('model_save_best_model'):
    os.mkdir('model_save_best_model')

In [ ]:
#Input layer
input_layer = Input(shape=(x_train.shape[1],))
#Dense hidden layer 1
layer1 = Dense(64,activation='relu',kernel_initializer=tf.keras.initializers.HeNormal())(input_layer)
#Dense hidden layer 2
layer2 = Dense(128,activation='relu',kernel_initializer=tf.keras.initializers.HeNormal())(layer1)
#Dense hidden layer 3
layer3 = Dense(256,activation='relu',kernel_initializer=tf.keras.initializers.HeNormal())(layer2)
#Dense hidden layer 4
layer4 = Dense(128,activation='relu',kernel_initializer=tf.keras.initializers.HeNormal())(layer3)
#Dense hidden layer 5
layer5 = Dense(64,activation='relu',kernel_initializer=tf.keras.initializers.HeNormal())(layer4)
#output layer
output = Dense(1,activation='sigmoid',kernel_initializer=tf.keras.initializers.RandomUniform(0,1))(layer5)
#Creating a model
model_best = Model(inputs=input_layer,outputs=output)

##Callbacks

In [ ]:
log_dir="/content/model_save_best_model/" 
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1, write_graph=True,write_grads=True)


filepath="model_save_best_model/weights-{epoch:02d}.hdf5"
checkpoint = ModelCheckpoint(filepath,monitor='val_accuracy',verbose=1)  #,save_best_only=True
# validation_accuracy learning rate
val_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.9,patience=0)
# lr change based on no.of epoch
lrschedule = LearningRateScheduler(changeLearningRate, verbose=1)
#earlystopping
earlystop = EarlyStopping(monitor='val_accuracy', patience=2, verbose=1)
#terminate on nan
term_nan = TerminateNaN()
# here we are creating a list with all the callbacks we want
callback_list = [val_lr,lrschedule, earlystop, checkpoint,term_nan,tensorboard_callback]

In [ ]:
opt = tf.keras.optimizers.Adam(0.01)

In [ ]:
model_best.compile(optimizer=opt, loss='binary_crossentropy',metrics=['accuracy',f1])

In [ ]:
history_best = model_best.fit(x_train,y_train,epochs=10, validation_data=(x_test,y_test),batch_size=16, callbacks=callback_list)

Epoch 1/10

Epoch 00001: LearningRateScheduler setting learning rate to 0.009999999776482582.
1000/1000 [==============================] - 6s 6ms/step - loss: 0.7026 - accuracy: 0.5081 - f1: 0.2856 - val_loss: 0.6942 - val_accuracy: 0.4992 - val_f1: 0.6554

Epoch 00001: saving model to model_save_best_model/weights-01.hdf5
Epoch 2/10

Epoch 00002: LearningRateScheduler setting learning rate to 0.009999999776482582.
1000/1000 [==============================] - 5s 5ms/step - loss: 0.6934 - accuracy: 0.4989 - f1: 0.3425 - val_loss: 0.6933 - val_accuracy: 0.4992 - val_f1: 0.6554

Epoch 00002: saving model to model_save_best_model/weights-02.hdf5
Epoch 3/10

Epoch 00003: LearningRateScheduler setting learning rate to 0.008549999631941318.
1000/1000 [==============================] - 5s 5ms/step - loss: 0.6934 - accuracy: 0.5001 - f1: 0.3312 - val_loss: 0.6932 - val_accuracy: 0.4992 - val_f1: 0.6554

Epoch 00003: saving model to model_save_best_model/weights-03.hdf5
Epoch 00003: early stoppi

##Tensorboard

In [ ]:
!kill 2790

In [1]:
# %tensorboard --logdir /content/model_save_relu

#Observations

* We have implemented 4 models as described in the task
* Early stopping is occuring in most cases as the alidation accuracy is not improving in more than 2 epochs (patience =2) . Though in one case, we can see the loss value turns out to be NaN and hence the TerminateOn NAN call back is triggered
* We can also observe the changes in acuuracy and other parameters in tensorboard for each epoch 
* The best model, with ADAM optimizer seems to have reached a similar accuracy of the other models in less number of iterations
* With this assignment, we've explored several callback that can be used to monitor the train phase in granular level like on end of epoch/start of it.